In [43]:
pip install -U datasets

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
from datasets import load_dataset
import pandas as pd


In [45]:
ds = load_dataset("Multilingual-Perspectivist-NLU/MultiPICo")

In [46]:
languages = set(ds['train']['language'])
print(languages)

{'nl', 'it', 'fr', 'de', 'pt', 'hi', 'ar', 'en', 'es'}


In [47]:
dataset = ds["train"]
df = dataset.to_pandas()

COLONNE df_ita \

"label" = label assegnata da ciascun annotatore al testo \
"timestamp" = data e ora di completamento dell'annotazione (a livello di testo) \
"level" = livello del commento (raramente ci sono commenti di secondo livello. In quel caso il rapporto tra post e reply rischia di essere meno chiaro) \
"source" = fonte del commento (reddit o twitter) \
"reply" = testo a cui gli annonatori hanno assegnato la label \
"post" = contesto della reply \
"language" = lingua del testo \
"language_variety" = varietà linguistica del testo \
"post_id" = id del post \
"majority_vote" = majority label \
\
\
DATI DEMOGRAFICI: \
'Age' = età di ogni annotatore (il task è stato completato nel 2023) \
'Gender' = genere binario \
'Ethnicity simplified' = etnia \
'Country of birth' = dove è nato \
'Country of residence' = dove risiede \
'Nationality' = nazionalità. Questa colonna è stata utilizzata per inferira la prima lingua dell'annotatore \
'Student status' = label binaria (Yes, No) \
'Employment status' = più opzioni possibili, con un buon grado di granularità \
'Generation' = Generazione di ciascun annotatore \
'GenerationAggregated' = binario (Young, Old) \
'Employed' = Employment status aggregato (Yes, No)



In [ ]:
lang = "it"

In [49]:
df_ita = df[df["language"] == lang]
df_ita["label"] = df["label"].astype(int)
df_ita.shape

C:\Users\micha\AppData\Local\Temp\ipykernel_15824\521875277.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ita["label"] = df["label"].astype(int)


(24036, 22)

In [50]:
# Majority vote function
def compute_majority(series):
    count_0 = (series == 0).sum()
    count_1 = (series == 1).sum()
    if count_0 > count_1:
        return 0
    elif count_1 > count_0:
        return 1
    else:
        return 'both'

# Compute majority per group and map back
majority_votes = df_ita.groupby('reply_id')['label'].apply(compute_majority)



In [51]:
df_ita['majority_vote'] = df_ita['reply_id'].map(majority_votes)

C:\Users\micha\AppData\Local\Temp\ipykernel_15824\188802293.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ita['majority_vote'] = df_ita['reply_id'].map(majority_votes)


-------------------------------------------
\
COLONNE df_ita_gold \

"level" = livello del commento (raramente ci sono commenti di secondo livello. In quel caso il rapporto tra post e reply rischia di essere meno chiaro) \
"source" = fonte del commento (reddit o twitter) \
"reply" = testo a cui gli annonatori hanno assegnato la label \
"post" = contesto della reply \
"language" = lingua del testo \
"language_variety" = varietà linguistica del testo \
"reply_id" = id della reply \
"post_id" = id del post \
"majority_vote" = majority label

In [52]:
df_ita_gold = df_ita.drop_duplicates(subset="reply_id")
df_ita_gold = df_ita_gold[['level', 'source', 'reply', 'post', 'language',
       'language_variety', 'reply_id',
       'post_id', 'majority_vote']]

print(df_ita_gold["majority_vote"].value_counts())

majority_vote
0       3265
1       1194
both     224
Name: count, dtype: int64


In [53]:
df_ita.to_csv(f"multipico_{lang}.csv")
df_ita_gold.to_csv(f"multipico_{lang}_gold.csv")

In [54]:
df_ita_gold

,level,source,reply,post,language,language_variety,reply_id,post_id,majority_vote
59697,1.0,reddit,El mapa dice que son las universidades fundada...,Quien quiera entender mejor lo que hicieron lo...,es,es,3971138134852140511,1459504691130595496,0
59698,1.0,reddit,Yo tambien soy colombiano en el exterior y tam...,"Querido amigo, no estoy en colombia actualment...",es,co,13166283860326302268,12854558575510118782,0
59699,1.0,twitter,"@USER Tranquilo te desmayas, cayese viejo lesb...",Mas rápido resolvió organizar una marcha que d...,es,mx,11283222104825576213,15307786463238434061,1
59700,1.0,reddit,De hace unos 15 años más o menos.,De hace cuanto es esta publicidad?,es,ar,5222616372363074128,3597601810208320049,0
59701,1.0,reddit,"Cierto, muy cierto asi mismo los grados Ferenh...",El sistema imperial. Vengo de un país donde se...,es,us,11837953237958371441,4358583015840553880,0
...,...,...,...,...,...,...,...,...,...
82260,1.0,twitter,"@USER Si, total la corte es de el y le falla s...",Mauricio Macri siempre respeto la división de ...,es,ar,16365657620940687090,7706938822044563404,1
82487,1.0,reddit,El tren es lento y viejo y llevan siglos inten...,"guiri que soy, me sorprende que los entiendo c...",es,es,12212217740734361440,13262766491695924928,1
82550,1.0,twitter,Obvio malas decisiones,No tomo nunca más,es,ar,7222828686183793720,5927788441106002645,1
82568,1.0,twitter,@USER @USER este man tiene un dicho peor que ...,A mí me causa mucha ternura ver como las incog...,es,co,13142093861103943766,762135156824764868,0


In [55]:
# Save the dataframe to a CSV file
df_ita_gold.to_csv(f'df_{lang}_gold.csv', index=False)

In [56]:
# prompt: save the csv if the rows have majority vote equals to 1 or both. print the number of the roews

# Filter rows where 'majority_vote' is 1 or 'both'
df_filtered = df_ita_gold[df_ita_gold['majority_vote'].isin([1, 'both'])]

# Print the number of rows in the filtered dataframe
print(f"Number of rows with majority vote 1 or both: {len(df_filtered)}")

# Save the filtered dataframe to a new CSV file
df_filtered.to_csv(f'multipico_{lang}_filtered.csv', index=False)

Number of rows with majority vote 1 or both: 1418
